##### Importing module

In [1]:
import numpy as np
import pandas as pd
import spacy 
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

### loading csv data 

In [2]:
df=pd.read_csv("ner.csv",index_col=False)

In [3]:
df.head(10)

,text,labels
0,Thousands of demonstrators have marched throug...,O O O O O O B-geo O O O O O B-geo O O O O O B-...
1,Iranian officials say they expect to get acces...,B-gpe O O O O O O O O O O O O O O B-tim O O O ...
2,Helicopter gunships Saturday pounded militant ...,O O B-tim O O O O O B-geo O O O O O B-org O O ...
3,They left after a tense hour-long standoff wit...,O O O O O O O O O O O
4,U.N. relief coordinator Jan Egeland said Sunda...,B-geo O O B-per I-per O B-tim O B-geo O B-gpe ...
5,Mr. Egeland said the latest figures show 1.8 m...,B-per I-per O O O O O O O O O O O O O O O O O ...
6,He said last week 's tsunami and the massive u...,O O O O O O O O O O O O O O O O O O B-geo O B-...
7,"Some 1,27,000 people are known dead .",O O O O O O O
8,"Aid is being rushed to the region , but the U....",O O O O O O O O O O B-geo O O O O O O O O O O ...
9,Lebanese politicians are condemning Friday 's ...,B-gpe O O O B-tim O O O O O O O O B-geo O O O ...


In [4]:
df.isnull().sum()

text      0
labels    0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47959 entries, 0 to 47958
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    47959 non-null  object
 1   labels  47959 non-null  object
dtypes: object(2)
memory usage: 749.5+ KB


In [6]:
df.shape

(47959, 2)

In [7]:
ls=list(df["text"])
ls2=list(df["labels"])

#### converting text data to this form  TRAIN_DATA = [[text, {"entities": [(start, end, label)]}]]

In [8]:
def punc(ls):
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for ele in ls:
        if ele in punc:
            ls = ls.replace(ele, "")
    return ls
def convert(ls, ls2):
    i = 0
    l1 = []

    while (i < len(ls) and i < len(ls2)):
        text1 = punc(ls[i])
        text2 = punc(ls2[i])
        sep1 = text1.split(" ")
        sep2 = text2.split(" ")
        j = 0
        start = 0
        l3 = []
        k = 0
        while (j < len(sep1) and j < len(sep2)):
            start = k
#             print(start)
            if (sep2[j] != "O"):
                ent = sep2[j]
                ent = "".join(ent)
                end = start + len(sep1[j])
                l3.append((start, end, ent[1:len(sep2[j])]))
            
#             print(len(sep1[j]))
            k = k + len(sep1[j])+1
            j = j + 1
#         text1=text1.split()
        if (len(l3) > 0):
            l1.append([text1.strip(), {"entities": (l3)}])

        i = i + 1
    train_data = list(l1)
    return train_data


### converting

In [9]:
train=convert(ls,ls2)

In [10]:
train[0:2]

[['Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country',
  {'entities': [(48, 54, 'geo'), (77, 81, 'geo'), (111, 118, 'gpe')]}],
 ['Iranian officials say they expect to get access to sealed sensitive parts of the plant Wednesday  after an IAEA surveillance system begins functioning',
  {'entities': [(0, 7, 'gpe'), (87, 96, 'tim'), (107, 111, 'org')]}]]

In [11]:
len(train)

40917

In [12]:
import numpy as np
import spacy 
import pickle
import json
from spacy.training.example import Example
from spacy.tokens import DocBin
from tqdm import tqdm
import random

#### dump train data into json

In [13]:
import json
with open('output1.json', 'w') as outfile:
    json.dump(train, outfile)

#### load json data

In [14]:
def load_data(file):
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)
data=load_data("output1.json")
train_data=data[0:30000]
test_data=data[30000:]

#### converting into spacy format

In [15]:
nlp=spacy.blank('en')
def create_traing(train_data):
    db=DocBin()
    for text,annot in tqdm(train_data):
        doc=nlp.make_doc(text)
        ent=[]
#         print(annot)
        for start,end,label in annot['entities']:
            span=doc.char_span(start,end,label=label,alignment_mode="strict")
#             print(label)
#             print(span)
            if span is None:
                continue
            else:
                ent.append(span)
        doc.ents=ent
        db.add(doc)
    return db

In [16]:
#####     debugging for refernce     ######
# for text,annot in tqdm(data[0:]):
#     print(annot)
#     doc=nlp.make_doc(text)
#     for start,end,label in annot["entities"]:
#         print(start,end,label)
#         print(text[start:end],label)
#         span=doc.char_span(start,end,label=label,alignment_mode="strict")
#         print(span)
train_data=create_traing(train_data)
train_data.to_disk("./train.spacy")
dev_data=create_traing(test_data)
dev_data.to_disk("./dev.spacy")
        

100%|███████████████████████████████████████████████████████████████████████████| 10917/10917 [00:16<00:00, 668.00it/s]


In [17]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy

[i] Using CPU


2021-07-21 14:57:29.689739: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2021-07-21 14:57:29.689780: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2021-07-21 14:57:35,244] [INFO] Set up nlp object from config
[2021-07-21 14:57:35,257] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-07-21 14:57:35,263] [INFO] Created vocabulary
[2021-07-21 14:57:35,263] [INFO] Finished initializing nlp object
[2021-07-21 14:58:56,742] [INFO] Initialized pipeline components: ['tok2vec', 'ner']



=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     41.35    9.11    5.48   26.88    0.09
  0     200         91.43   3705.62   61.05   63.20   59.05    0.61
  0     400        179.25   2405.99   76.00   76.04   75.96    0.76
  0     600        212.73   2649.04   79.79   80.61   78.99    0.80
  0     800        223.31   2796.84   81.56   82.12   81.02    0.82
  0    1000        270.77   3136.67   82.81   83.12   82.51    0.83
  0    1200        328.92   3821.08   83.27   83.67   82.87    0.83
  0    1400        368.06   4298.29   83.93   84.66   83.21    0.84
  0    1600        466.27   4996.56   84.47   85.86   83.13    0.84
  0   

#### load Custom NER model for checking its working fine or not

In [7]:
nlp1 = spacy.load(R"C:\Users\Ankush babu\Desktop\Travel insurance\custom on new dataset\output\model-best") #load the best model
doc = nlp1("President Bush has signed legislation that will require screening of all air and sea cargo  and will provide more money to cities deemed to be at high risk of a terrorist attack in California")
spacy.displacy.render(doc, style="ent",jupyter=True)